In [6]:
#importing libraries
import cv2
import time
import math
import mediapipe as mp

#capturing the video
cap=cv2.VideoCapture(0)

#array to store the tuple of coordinates of tip of the index finger
centre_x=[0]
centre_y=[0]

dip_x=0
dip_y=0

wrist_x=0
wrist_y=0

tip_x=0
tip_x=0

ring_x=0
ring_y=0

#this will help us detect the hand and draw the reference points
mphand=mp.solutions.hands
hands=mphand.Hands(max_num_hands=1)
mpDraw=mp.solutions.drawing_utils    #this statement will help us plot the 21 reference points and their connections

#variables required to print frame rate
previous_time=0
current_time=0

while True:
    success, img=cap.read()
    
    #converting image to rgb
    img_rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #detecting hand/s using 21 points of reference, these reference points will be plotted on the hand
    result=hands.process(img_rgb)
    
    if result.multi_hand_landmarks:
        for handlms in result.multi_hand_landmarks:    #loop: handlms gives us the number of hands i.e. 1 hand, 2hands...
            for id, lm in enumerate(handlms.landmark):    #loop: gives us individual reference point on each hand
                h, w, c=img.shape
                cx, cy=int(math.floor(lm.x*w)), int(math.floor(lm.y*h))    #give us the coordinates for the reference points
                if id==0:
                    wrist_x=cx
                    wrist_y=cy                    
                elif id==12:
                    dip_x=cx
                    dip_y=cy
                elif id==16:
                    i=0
                    ring_x=cx
                    ring_y=cy
                    if math.dist([tip_x, tip_y], [wrist_x, wrist_y])-30<math.dist([ring_x, ring_y], [wrist_x, wrist_y]):
                        cv2.circle(img, (cx, cy), 50, (0, 0, 0), cv2.FILLED)
                        for temp_x, temp_y in zip(centre_x, centre_y):
                            if math.dist([temp_x, temp_y], [ring_x, ring_y])<50:
                                centre_x[i]=w+1
                                centre_y[i]=h+1
                            i+=1
                elif id==8:
                    tip_x=cx
                    tip_y=cy
                    if math.dist([tip_x, tip_y], [wrist_x, wrist_y])>math.dist([dip_x, dip_y], [wrist_x, wrist_y]):
                        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                        centre_x.append(int(cx))
                        centre_y.append(int(cy))
                    else:
                        if (centre_x[-1], centre_y[-1])!=(w+1, h+1):
                            centre_x.append(w+1)
                            centre_y.append(h+1)
            mpDraw.draw_landmarks(img, handlms, mphand.HAND_CONNECTIONS)
            #the above statement will actually plot the 21 reference points and their connections
            
    #calculating the frame rate
    current_time=time.time()
    frame_rate=1/(current_time-previous_time)
    previous_time=current_time
    
    #draw according to movement of given index
    for i in range(1, len(centre_x)):
        if centre_x[i]<w+1 and centre_y[i]<h+1 and centre_x[i-1]<w+1 and centre_y[i-1]<h+1:
            cv2.line(img, (centre_x[i-1], centre_y[i-1]), (centre_x[i], centre_y[i]), (255, 0, 255), 3)
    
    #displaying the results
    img=cv2.flip(img,1)
    cv2.putText(img, str(int(frame_rate)), (20, 40), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
    cv2.imshow("camera", img)
    cv2.waitKey(1)

KeyboardInterrupt: 